# Compare strain growth versus titers

In [1]:
######## snakemake preamble start (automatically inserted, do not edit) ########
import sys;sys.path.extend(['/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/lib/python3.12/site-packages', '/fh/fast/bloom_j/computational_notebooks/jbloom/2024/flu_H3_2023-2024_strains_MLR', '/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/bin', '/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/lib/python3.12', '/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/lib/python3.12/lib-dynload', '/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/lib/python3.12/site-packages', '/fh/fast/bloom_j/software/miniforge3/envs/flu_H3_2023-2024_strains_MLR/lib/python3.12/site-packages/setuptools/_vendor', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmp6hdnc9wr/file/fh/fast/bloom_j/computational_notebooks/jbloom/2024/flu_H3_2023-2024_strains_MLR/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2024/flu_H3_2023-2024_strains_MLR/notebooks']);import pickle;from snakemake import script;script.snakemake = pickle.loads(b'\x80\x04\x95\x13\n\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8cEresults/mlr/growth_advantages_gisaid-ha1-within1_2023-mincounts50.csv\x94\x8c\x1adata/H3_seqneut_titers.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x06growth\x94K\x00N\x86\x94\x8c\x06titers\x94K\x01N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x15h\x06\x8c\x0eAttributeGuard\x94\x93\x94)\x81\x94}\x94\x8c\x04name\x94h\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sbh\x0fh\nh\x11h\x0bub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8cRresults/growth_vs_titers/growth_vs_titers_gisaid-ha1-within1_2023-mincounts50.html\x94a}\x94(h\r}\x94\x8c\x05chart\x94K\x00N\x86\x94sh\x13]\x94(h\x15h\x16eh\x15h\x18)\x81\x94}\x94h\x1bh\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sbh%h"ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94}\x94(h\r}\x94h\x13]\x94(h\x15h\x16eh\x15h\x18)\x81\x94}\x94h\x1bh\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sbub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94(\x8c\x12gisaid-ha1-within1\x94\x8c\x102023-mincounts50\x94e}\x94(h\r}\x94(\x8c\x07protset\x94K\x00N\x86\x94\x8c\x06mlrfit\x94K\x01N\x86\x94uh\x13]\x94(h\x15h\x16eh\x15h\x18)\x81\x94}\x94h\x1bh\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sb\x8c\x07protset\x94h;\x8c\x06mlrfit\x94h<ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\r}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x13]\x94(h\x15h\x16eh\x15h\x18)\x81\x94}\x94h\x1bh\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sbhRK\x01hTK\x01hVhOub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8cSresults/growth_vs_titers/growth_vs_titers_gisaid-ha1-within1_2023-mincounts50.ipynb\x94a}\x94(h\r}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x13]\x94(h\x15h\x16eh\x15h\x18)\x81\x94}\x94h\x1bh\x15sbh\x16h\x18)\x81\x94}\x94h\x1bh\x16sbhdhaub\x8c\x06config\x94}\x94(\x8c\x0cstrain_prots\x94\x8c(data/H3_library_constructs_protein.fasta\x94\x8c\x08protsets\x94}\x94(\x8c\x12gisaid-ha1-within1\x94}\x94(\x8c\x07protset\x94\x8c\x1bdata/gisaid_flu_h3_prots.fa\x94\x8c\x04trim\x94]\x94(K\x14M\\\x01e\x8c\x07maxdiff\x94K\x01u\x8c\x10gisaid-ha1-exact\x94}\x94(\x8c\x07protset\x94\x8c\x1bdata/gisaid_flu_h3_prots.fa\x94\x8c\x04trim\x94]\x94(K\x14M\\\x01e\x8c\x07maxdiff\x94K\x00u\x8c\x19gisaid-ectodomain-within1\x94}\x94(\x8c\x07protset\x94\x8c\x1bdata/gisaid_flu_h3_prots.fa\x94\x8c\x04trim\x94]\x94(K\x14M\x15\x02e\x8c\x07maxdiff\x94K\x01uu\x8c\x07mlrfits\x94}\x94(\x8c\x112023-mincounts100\x94}\x94(\x8c\ndate_start\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe7\x01\x01\x94\x85\x94R\x94\x8c\x08date_end\x94h\x8dC\x04\x07\xe7\x0c\x1f\x94\x85\x94R\x94\x8c\nmin_counts\x94Kd\x8c\x13keep_not_in_library\x94\x89\x8c\x18keep_insufficient_counts\x94\x89u\x8c\x1b2023-mincounts100-keepother\x94}\x94(\x8c\ndate_start\x94h\x8dC\x04\x07\xe7\x01\x01\x94\x85\x94R\x94\x8c\x08date_end\x94h\x8dC\x04\x07\xe7\x0c\x1f\x94\x85\x94R\x94\x8c\nmin_counts\x94Kd\x8c\x13keep_not_in_library\x94\x88\x8c\x18keep_insufficient_counts\x94\x88u\x8c\x102023-mincounts50\x94}\x94(\x8c\ndate_start\x94h\x8dC\x04\x07\xe7\x01\x01\x94\x85\x94R\x94\x8c\x08date_end\x94h\x8dC\x04\x07\xe7\x0c\x1f\x94\x85\x94R\x94\x8c\nmin_counts\x94K2\x8c\x13keep_not_in_library\x94\x89\x8c\x18keep_insufficient_counts\x94\x89u\x8c\x1dmid2022early2024-mincounts100\x94}\x94(\x8c\ndate_start\x94h\x8dC\x04\x07\xe6\x07\x01\x94\x85\x94R\x94\x8c\x08date_end\x94h\x8dC\x04\x07\xe8\x03\x1f\x94\x85\x94R\x94\x8c\nmin_counts\x94Kd\x8c\x13keep_not_in_library\x94\x89\x8c\x18keep_insufficient_counts\x94\x89u\x8c\x172022to2024-mincounts100\x94}\x94(\x8c\ndate_start\x94h\x8dC\x04\x07\xe6\x01\x01\x94\x85\x94R\x94\x8c\x08date_end\x94h\x8dC\x04\x07\xe8\x08\x01\x94\x85\x94R\x94\x8c\nmin_counts\x94Kd\x8c\x13keep_not_in_library\x94\x89\x8c\x18keep_insufficient_counts\x94\x89uu\x8c\x0cmlr_settings\x94}\x94(\x8c\x0cpivot_strain\x94\x8c\x18A/Singapore/NUH0526/2023\x94\x8c\x16plot_window_frame_days\x94K\x0e\x8c\x07mlr_tau\x94G@\x0c\xcc\xcc\xcc\xcc\xcc\xcd\x8c\nnum_warmup\x94M\xe8\x03\x8c\x0bnum_samples\x94M\xe8\x03\x8c\x0chpd_interval\x94K_u\x8c\x06titers\x94\x8c\x1adata/H3_seqneut_titers.csv\x94u\x8c\x04rule\x94\x8c\x10growth_vs_titers\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c[/fh/fast/bloom_j/computational_notebooks/jbloom/2024/flu_H3_2023-2024_strains_MLR/notebooks\x94ub.');del script;from snakemake.logging import logger;from snakemake.script import snakemake; logger.printshellcmds = False;import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2024/flu_H3_2023-2024_strains_MLR');
######## snakemake preamble end #########


In [2]:
# get variables from snakemake

desc = f"{snakemake.wildcards.protset}_{snakemake.wildcards.mlrfit}"
print(desc)

growth_csv = snakemake.input.growth
titers_csv = snakemake.input.titers

chart_html = snakemake.output.chart

corr_titer_cutoff_range = [40, 1000]
corr_titer_cutoff_points = 100  # logarithmically spaced

nrandom = 20

gisaid-ha1-within1_2023-mincounts50


In [ ]:
import altair as alt

import numpy

import pandas as pd

import scipy

_ = alt.data_transformers.disable_max_rows()

Read growth, then get titers just for those strains with growth estimates:

In [ ]:
growth = pd.read_csv(growth_csv)

titers = (
    pd.read_csv(titers_csv)
    .rename(columns={"virus": "strain"})
    [["strain", "serum", "titer"]]
    .assign(strain=lambda x: x["strain"].map(lambda s: 'A/Singapore/NUH0526/2023' if s == 'A/Massachusetts/18/2022' else s))
)
assert set(growth["strain"]).issubset(titers["strain"])
assert len(titers) == len(titers.groupby(["strain", "serum"]))

print(
    f"Read titers for {titers['strain'].nunique()} strains, of which "
    f"{growth['strain'].nunique()} have growth advantage estimates."
)
nsera = titers["serum"].nunique()

titers = titers[titers["strain"].isin(set(growth["strain"]))]

Get the correlation of growth with the **geometric** mean titer, median titer, and the fraction of sera below whatever cutoff gives teh best correlation.
Do this for the actual data, and randomizations of the data (randomizing the growth rates among strains) to get P-values.
Some notes:
 - Correlations with mean and median titers are with the **log** of these titers, whereas with fraction sera below cutoff it is with the value
 - The P-values are one-sided, testing if mean and median titers are negatively correlated with growth, and fraction sera below cutoff is positive correlated.

In [ ]:
def get_corrs_by_cutoff(
    growth_df,
    titers_df,
    cutoff_range=corr_titer_cutoff_range,
    cutoff_points=corr_titer_cutoff_points,
):
    """For each titer cutoff, get Pearson correlation of growth w frac titers below."""
    assert cutoff_range[0] < cutoff_range[1]
    corrs = []
    for cutoff in numpy.logspace(
        numpy.log10(cutoff_range[0]), numpy.log10(cutoff_range[1]), cutoff_points
    ):
        frac_below = (
            titers_df
            .groupby("strain", as_index=False)
            .aggregate(frac=pd.NamedAgg("titer", lambda s: (s <= cutoff).sum() / len(s)))
            .merge(growth_df[["strain", "growth_advantage_median"]])
        )
        if frac_below["frac"].nunique() == 1:
            continue  # cannot compute correlation if values all identical
        r = scipy.stats.pearsonr(frac_below["growth_advantage_median"], frac_below["frac"])[0]
        corrs.append((cutoff, r))
    return pd.DataFrame(corrs, columns=["titer_cutoff", "correlation"])

# get actual correlations
corr_cutoff_actual = get_corrs_by_cutoff(growth, titers)
best_cutoff = corr_cutoff_actual.sort_values("correlation")["titer_cutoff"].tolist()[-1]
corr_cutoffs = [corr_cutoff_actual.assign(dataset="actual")]
growth_actual_scatter = growth.merge(
    titers.groupby("strain", as_index=False).aggregate(
        mean_titer=pd.NamedAgg("titer", lambda s: scipy.stats.gmean(s)),
        median_titer=pd.NamedAgg("titer", "median"),
        frac_below_titer=pd.NamedAgg("titer", lambda s: (s <= best_cutoff).sum() / len(s)),
    ),
    on="strain",
    validate="one_to_one",
)

# get correlations for randomizations
randomized_corrs = {"mean_titer": [], "median_titer": [], "frac_below_titer": []}
for irandom in range(nrandom):
    numpy.random.seed(irandom)
    growth_random = growth.assign(
        growth_advantage_median=lambda x: numpy.random.permutation(
            x["growth_advantage_median"].values
        )
    )
    corr_cutoffs.append(
        get_corrs_by_cutoff(growth_random, titers).assign(dataset=f"random_{irandom}")
    )
    assert (growth_actual_scatter["strain"] == growth_random["strain"]).all()
    for col in ["mean_titer", "median_titer"]:
        randomized_corrs[col].append(
            scipy.stats.pearsonr(
                growth_random["growth_advantage_median"],
                numpy.log10(growth_actual_scatter[col]),
            )[0]
        )
    randomized_corrs["frac_below_titer"].append(corr_cutoffs[-1]["correlation"].max())
    
corr_cutoffs = pd.concat(corr_cutoffs, ignore_index=True)

corrs_and_pvals = []
for var in ["mean_titer", "median_titer", "frac_below_titer"]:
    corr = scipy.stats.pearsonr(
        growth_actual_scatter["growth_advantage_median"],
        numpy.log10(growth_actual_scatter[var]),
    )[0]
    if var == "frac_below_titer":
        p = sum(r >= corr for r in randomized_corrs[var]) / nrandom
    else:
        p = sum(r <= corr for r in randomized_corrs[var]) / nrandom
    corrs_and_pvals.append((var, corr, p))

corrs_and_pvals = pd.DataFrame(corrs_and_pvals, columns=["stat", "correlation", "P"]).assign(
    p_str=lambda x: x["P"].map(
        lambda p: f"P = {p:.2g}" if p else f"P < {1 / nrandom:.2g}"
    ),
    label=lambda x: x.apply(
        lambda r: f"R = {r['correlation']:.2f} ({r['p_str']})",
        axis=1,
    ),
)

Plot correlation of growth with fraction of titers above a cutoff for the actual data and the randomizations:

In [ ]:
corr_cutoffs_chart = (
    alt.Chart(corr_cutoffs)
    .transform_calculate(actual=alt.datum["dataset"] == "actual")
    .encode(
        alt.X(
            "titer_cutoff",
            scale=alt.Scale(zero=False, nice=False, type="log", padding=5),
            title="neutralization titer cutoff",
            axis=alt.Axis(grid=False, labelOverlap=True),
        ),
        alt.Y("correlation", axis=alt.Axis(grid=False)),
        alt.Color(
            "actual:N",
            scale=alt.Scale(domain=[True, False], range=["black", "slateblue"]),
            legend=None,
        ),
        alt.Opacity(
            "actual:N",
            scale=alt.Scale(domain=[True, False], range=[1, 0.15]),
        ),
        alt.StrokeWidth(
            "actual:N",
            scale=alt.Scale(domain=[True, False], range=[2, 1]),
        ),
        alt.Detail("dataset"),
        tooltip=[
            "dataset",
            alt.Tooltip("titer_cutoff", format=".1f"),
            alt.Tooltip("correlation", format=".2f"),
        ],
    )
    .mark_line()
    .properties(
        width=250,
        height=170,
        title=alt.TitleParams(
            [
                "correlation of strain growth with",
                "fraction sera with titers below cutoff",
            ],
            subtitle=[
                "thick black line is actual data; thin",
                f"blue lines are {nrandom} randomizations",
            ],
        ),
    )
)

corr_cutoffs_chart

Plot the correlation scatter plots:

In [ ]:
corr_base = alt.Chart(
    growth_actual_scatter.assign(
        ga_label=lambda x: x.apply(
            lambda r: f"{r['growth_advantage_median']:.2f} [{r['growth_advantage_hpd_min']:.2f} - {r['growth_advantage_hpd_max']:.2f}]",
            axis=1,
        )
    )
)

strain_sel = alt.selection_point(fields=["strain"], on="mouseover", empty=False)

corr_charts = []
first = True
for stat in ["mean_titer", "median_titer", "frac_below_titer"]:
    stat_name = {
        "mean_titer": "mean titer",
        "median_titer": "median titer",
        "frac_below_titer": f"fraction sera with titer < {best_cutoff:.0f}",
    }[stat]

    corr_scatter = (
        corr_base
        .encode(
            alt.X(
                stat,
                title=stat_name,
                scale=alt.Scale(
                    zero=False,
                    nice=False,
                    padding=6,
                    type="linear" if stat == "frac_below_titer" else "log",
                ),
                axis=alt.Axis(grid=False, labelOverlap=True),
            ),
            alt.Y(
                "growth_advantage_median",
                title="growth advantage" if first else None,
                scale=alt.Scale(zero=False, nice=False, padding=6),
                axis=alt.Axis(grid=False, labels=first),
            ),
            stroke=alt.condition(strain_sel, alt.value("red"), alt.value("black")),
            strokeWidth=alt.condition(strain_sel, alt.value(2), alt.value(1)),
            tooltip=[
                "strain",
                alt.Tooltip("ga_label", title="growth advantage"),
                alt.Tooltip(stat, title=stat_name, format=".2f"),
            ],
        )
        .mark_circle(size=45, color="black", strokeOpacity=1, fillOpacity=0.5)
        .properties(width=150, height=150)
    )
    corr_errorbar = (
        corr_base
        .encode(
            alt.X(stat),
            alt.Y("growth_advantage_hpd_min"),
            alt.Y2("growth_advantage_hpd_max"),
        )
        .mark_rule(color="black")
    )
    corr_text = (
        alt.Chart(corrs_and_pvals[corrs_and_pvals["stat"] == stat])
        .encode(
            x=alt.value(2) if stat == "frac_below_titer" else alt.value(148),
            y=alt.value(2),
            text="label",
        )
        .mark_text(
            baseline="top",
            align="left" if stat == "frac_below_titer" else "right",
            color="blue",
        )
    )

    corr_charts.append(corr_scatter + corr_errorbar + corr_text)

    first = False

corr_chart = alt.hconcat(*corr_charts, spacing=7).add_params(strain_sel).properties(
    title=alt.TitleParams(
        f"strain growth versus neutralization by {nsera} sera",
        anchor="middle",
    ),
)

corr_chart

Make and save the overall chart:

In [ ]:
chart = alt.vconcat(corr_chart, corr_cutoffs_chart, spacing=15).properties(
    title=alt.TitleParams(
        f"strain growth vs neutralization for {desc}",
        anchor="middle",
        fontSize=15,
        dy=-20,
    )
)

print(f"Saving to {chart_html}")
chart.save(chart_html)

chart